# Import Libraries

In [59]:
params = {}

In [60]:
import time
import json
import sys

from data_task_helpers import something

something()

from src.dte.data_task_tools import assert_dte_tools_available, get_resolved_parameters_for_connection, initialise_data_task, find_json_arg  # noqa: E402

something
['/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace/.venv/lib/python3.11/site-packages', '/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace/duft', '/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace/duft-server']


In [61]:
   
environment = initialise_data_task("Jupyter Sample Data Task", params=params)
params["name"] = params.get("customname", params.get("name", "No parameters given!"))
params["sleep_time"] = params.get("sleep_time", 0.2)

if not params:
    environment.log_error("No parameters given!")
    

DUFT Config: duft-config-namibia
DUFT Config: duft-config-namibia
[2024-08-13T12:19:30.262789] Jupyter Sample Data Task - INFO: Jupyter Sample Data Task initialised running in active mode
Jupyter Sample Data Task: Jupyter Sample Data Task initialised running in active mode


# Import dependancies

In [62]:
import jaydebeapi
import polars as pl
from pandas import DataFrame
from sqlalchemy import create_engine
from jaydebeapi import Connection
import time
from datetime import datetime

# Set Source and Destinaton Database Connections 

In [63]:
source_environment = get_resolved_parameters_for_connection("EPMS_Source")
destination_environment = get_resolved_parameters_for_connection("EPMS_Destination")

DUFT Config: duft-config-namibia
DUFT Config: duft-config-namibia


# Set Filemaker Driver Files

In [64]:
driver_jar_path = "/Library/Java/Extensions/fmjdbc.jar"
driver_class = "com.filemaker.jdbc.Driver"

# Process File Maker Data

In [65]:
def open_file_maker_connection():
    try:
        server = source_environment.get("server")
        username = source_environment.get("username")
        password = source_environment.get("password")
        return jaydebeapi.connect(driver_class, server, [username, password], driver_jar_path)
    except Exception as ex:
        return None


def close_file_maker_connection(con: Connection):
    if con is not None:
        con.close()


def fetch_file_maker_data(query: str, con: Connection):
    cursor = con.cursor()
    cursor.execute(query)
    data_rows = cursor.fetchall()
    cursor.close()
    return data_rows

# Open connection to FM

In [66]:
fm_connection = open_file_maker_connection()

# Create dim_patient Data Frame

In [67]:
patient_confs = {
    "query": "SELECT  r.name, d.name, f.name, f.code, p.id, p.sex, p.dob, p.maritalStatus, p.patientStatus, p.clientCode, pharmacyNumber, pmtctNumber, nameFirst, nameLast, phoneCellNumber, p.ageYear, resCurrentAddTown, resCurrentAddConstituency, resCurrentAddStreet, resPermanentAddTown, resPermanentAddConstituency, resPermanentAddStreet, idFacilityCurrent,  homeBasedCareOrg, homeBasedCareCode,  deathDate  FROM pat p inner join fac f on p.idFacilityCurrent = f.id  inner join region r on f.idregion=r.id inner join district d on f.idDistrict = d.id",
    "cols": ['region','district','current_facility', 'facility_code','client_id', 'sex', 'date_of_birth', 'marital_status','patient_status','client_code', 'pharmacy_code','pmtct_number', 'first_name','last_name','contact_number','age','current_town','current_constituency','current_street', 'permanent_town','permanent_constituency','permanent_street','id_facility_current', 'cbart_cargs_name', 'cbart_cargs_code', 'death_date']
}

patient_query = patient_confs.get("query")
patient_cols = patient_confs.get("cols")

patient_data = fetch_file_maker_data(patient_query, fm_connection)
dim_pat_df = pl.DataFrame(patient_data, schema=patient_cols)

dim_pat_df = dim_pat_df.with_columns([
    pl.col('date_of_birth').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('date_of_birth'),
    pl.lit('FName').alias('first_name'),
    pl.lit('LName').alias('last_name'),
    pl.lit('123456').alias('contact_number')
])

dim_pat_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,current_town,current_constituency,current_street,permanent_town,permanent_constituency,permanent_street,id_facility_current,cbart_cargs_name,cbart_cargs_code,death_date
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,None,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,None,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,None,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,None,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,2007-12-31
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,WINDHOEK,Katutura Central,ERF 54 WANAHEDA,WINDHOEK,Katutura Central,ERF 54 WANAHEDA,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,OMWENE TUUDA,None,None


# Get patient latest transfer status

In [68]:
fact_tsfr_confs = {
    "query": "SELECT idpatient,  status, \"date\" FROM tsfr",
    "cols": ['client_id' ,'transfer_status', 'transfer_date']
}

fact_tsfr_query = fact_tsfr_confs.get("query")
fact_tsfr_cols = fact_tsfr_confs.get("cols")

fact_tsfr_data = fetch_file_maker_data(fact_tsfr_query, fm_connection)
fact_tsfr_df = pl.DataFrame(fact_tsfr_data, schema=fact_tsfr_cols)

fact_tsfr_df = fact_tsfr_df.dropna(subset=['transfer_date'])

fact_tsfr_df = fact_tsfr_df.loc[fact_tsfr_df.groupby('client_id')['transfer_date'].idxmin()].reset_index(drop=True)
fact_tsfr_df.head()

fact_tsfr_df.head(100)

,client_id,transfer_status,transfer_date
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,Out,2018-01-05
1,0003CE1A-FDA7-0644-AC8A-6A2AC22F458C,Out,2009-06-01
2,000C0885-8782-1047-9575-2096412DF234,Out,2015-11-04
3,000FC632-25F4-F446-B53B-79332149CDE0,Out,2010-10-13
4,00148C55-3E2C-264B-86A9-213C01DBD15E,Out Pending,2024-01-22
...,...,...,...
95,02CB134D-12C7-9D43-A912-772007E0A5C2,In,2017-06-27
96,02CC9659-BC19-4B46-91C1-8D48653FC02E,In,2011-01-03
97,02CD8D38-5A84-124D-B51E-6E302BB7C5DE,Out,2007-05-15
98,02D07FE0-C4BE-924E-9936-F1F6F069AE54,In,2021-01-21


# Add transfer status to patient

In [69]:
dim_pat_df = dim_pat_df.join(fact_tsfr_df, on='client_id', how='left')

dim_pat_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,current_street,permanent_town,permanent_constituency,permanent_street,id_facility_current,cbart_cargs_name,cbart_cargs_code,death_date,transfer_status,transfer_date
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,NaN,NaN
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,NaN,NaN
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,None,NaN,NaN
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,None,None,None,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,None,2007-12-31,NaN,NaN
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,ERF 54 WANAHEDA,WINDHOEK,Katutura Central,ERF 54 WANAHEDA,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,OMWENE TUUDA,None,None,NaN,NaN


# Get first Weight and WHO Stage

In [70]:
fact_meas_confs = {
    "query": "SELECT idpatient, weight, whoStage,\"Date\" FROM Meas m",
    "cols": ['client_id', 'weight','who_stage', 'creation_date']
}

fact_meas_query = fact_meas_confs.get("query")
fact_meas_cols = fact_meas_confs.get("cols")

fact_meas_data = fetch_file_maker_data(fact_meas_query, fm_connection)
fact_meas_df = pl.DataFrame(fact_meas_data, schema=fact_meas_cols)

fact_meas_df = fact_meas_df.dropna(subset=['creation_date'])

fact_meas_df = fact_meas_df.loc[fact_meas_df.groupby('client_id')['creation_date'].idxmin()].reset_index(drop=True)
fact_meas_df.head()

fact_meas_df.head(100)

,client_id,weight,who_stage,creation_date
0,000150F9-2398-3047-8E7E-BBFFA6E1F4E1,0.0,1,2014-03-18
1,000C0885-8782-1047-9575-2096412DF234,0.0,2,2011-06-16
2,00148C55-3E2C-264B-86A9-213C01DBD15E,0.0,1,2005-05-09
3,001946D5-919B-CB43-9B0E-68C03FF5D595,0.0,None,2010-06-24
4,002D86C5-77BF-184E-A9E4-AFBD55EEA37A,56.8,1,2018-07-30
...,...,...,...,...
95,02AB4B71-C650-D147-A789-D32ED3A6D813,58.2,1,2018-08-20
96,02B17343-92BE-024F-AEBE-4EEAEF6A2163,47.0,1,2019-06-11
97,02B30FFB-8642-0D41-BE3E-5F72B8C698F9,84.0,1,2018-10-10
98,02BADAAF-A923-D445-AC46-290BEDBE6610,0.0,2,2017-12-28


# Create fact_hiv_diagnosis Data Frame

In [71]:
fact_hiv_diagnosis_confs = {
    "query": "SELECT idpatient, hivconfirmationdate, hivconfirmatoryresultsdate, hivconfirmatoryresultstype, idfacilitycreate, fullDisclosureDAte FROM Cd",
    "cols": ['client_id', 'hiv_confirmation_date','hiv_confirmatory_result_date', 'hiv_confirmatory_result_type', 'hiv_diagnosis_facility_id','full_disclosure_date']
}

fact_hiv_diagnosis_query = fact_hiv_diagnosis_confs.get("query")
fact_hiv_diagnosis_cols = fact_hiv_diagnosis_confs.get("cols")

fact_hiv_diagnosis_data = fetch_file_maker_data(fact_hiv_diagnosis_query, fm_connection)
fact_hiv_diagnosis_df = pl.DataFrame(fact_hiv_diagnosis_data, schema=fact_hiv_diagnosis_cols)

fact_hiv_diagnosis_df = fact_hiv_diagnosis_df.with_columns([
    pl.col('hiv_confirmation_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('hiv_confirmation_date'),
    pl.col('hiv_confirmatory_result_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('hiv_confirmatory_result_date')
])

fact_hiv_diagnosis_df = fact_hiv_diagnosis_df.groupby('client_id')['hiv_confirmation_date'].idxmin()

fact_hiv_diagnosis_df.head()

,client_id,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,full_disclosure_date
0,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,NaT,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
1,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,1996-09-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
2,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,2001-09-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
3,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,1997-10-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
4,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,1999-03-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None


# Add Initial weight and WHO Stage to fact_hiv_diagnosis

In [72]:
fact_meas_df_selected = fact_meas_df.select(['client_id', 'weight', 'who_stage'])

fact_hiv_diagnosis_df = fact_hiv_diagnosis_df.join(fact_meas_df_selected, on='client_id', how='left')

fact_hiv_diagnosis_df.head()

,client_id,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,full_disclosure_date,weight,who_stage
0,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,NaT,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
1,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,1996-09-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
2,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,2001-09-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
3,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,1997-10-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
4,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,1999-03-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,70.2,1


# Add dim_patient, fact_hiv_diagnosis to fact_sentinel_event dataframe

In [73]:
fact_sentinel_event_df = dim_pat_df.join(fact_hiv_diagnosis_df, on='client_id', how='left')

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,death_date,transfer_status,transfer_date,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,full_disclosure_date,weight,who_stage
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,None,NaN,NaN,NaT,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,None,NaN,NaN,1996-09-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,None,NaN,NaN,2001-09-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,2007-12-31,NaN,NaN,1997-10-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,NaN,NaN
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,None,NaN,NaN,1999-03-15,NaT,None,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None,70.2,1


# Create hiv_enrolment dataframe 

In [74]:
fact_hiv_enrolment_confs = {
    "query": "SELECT c.idpatient, c.idfacilitycreate, c.hivenrolleddate, c.disclosureenrollmentdate, c.arteligiblereason, c.artstartdate, p.artnumber, p.artnumber, p.artnumberlegacy, c.idfacilityARTStart, tbScreenResult FROM pat p LEFT JOIN Cd c ON p.id=c.idpatient LEFT JOIN ia a  ON p.id=a.idpatient",
    "cols": ['client_id', 'hiv_enrollment_facility_id', 'hiv_enrollment_date', 'hiv_disclosure_enrollment_date','art_eligible_reason','art_start_date', 'quantum_number' , 'art_number', 'art_number_legacy', 'arv_initiating_facility','tb_screen_result']
}


fact_hiv_enrolment_query = fact_hiv_enrolment_confs.get("query")
fact_hiv_enrolment_cols = fact_hiv_enrolment_confs.get("cols")

fact_hiv_enrolment_data = fetch_file_maker_data(fact_hiv_enrolment_query, fm_connection)
fact_hiv_enrolment_df = pl.DataFrame(fact_hiv_enrolment_data, schema=fact_hiv_enrolment_cols)

fact_hiv_enrolment_df = fact_hiv_enrolment_df.with_columns([
    pl.col('hiv_enrollment_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('hiv_enrollment_date'),
    pl.col('hiv_disclosure_enrollment_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('hiv_disclosure_enrollment_date'),
    pl.col('art_start_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('art_start_date')
])

fact_hiv_enrolment_df.head()

,client_id,hiv_enrollment_facility_id,hiv_enrollment_date,hiv_disclosure_enrollment_date,art_eligible_reason,art_start_date,quantum_number,art_number,art_number_legacy,arv_initiating_facility,tb_screen_result
0,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-07-24,NaT,Clinical & CD4,2004-03-25,10310-0703-72239,10310-0703-72239,115070300026,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
1,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-09-02,NaT,Clinical only,2003-09-18,10310-0903-72240,10310-0903-72240,115090300076,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
2,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-08-28,NaT,CD4 # or %,2003-08-28,10310-0803-72241,10310-0803-72241,115080300051,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
3,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-08-12,NaT,CD4 # or %,2003-08-12,10310-0803-72242,10310-0803-72242,115080300052,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
4,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-07-08,NaT,Clinical & CD4,2003-08-22,10310-0703-72243,10310-0703-72243,115070300002,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,Incomplete


# Add fact_hiv_enrolment to fact_sentinel_event dataframe

In [75]:
fact_sentinel_event_df = fact_sentinel_event_df.join(fact_hiv_enrolment_df, on='client_id', how='left')
fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,hiv_enrollment_facility_id,hiv_enrollment_date,hiv_disclosure_enrollment_date,art_eligible_reason,art_start_date,quantum_number,art_number,art_number_legacy,arv_initiating_facility,tb_screen_result
0,Khomas,Windhoek,Katutura Intermediate Hospital,10310,EDC01C7E-D62B-574A-8A82-08E0182DC6E6,Female,1965-06-17,Single,Lost,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-07-24,NaT,Clinical & CD4,2004-03-25,10310-0703-72239,10310-0703-72239,115070300026,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
1,Khomas,Windhoek,Katutura Intermediate Hospital,10310,1E6B7D7F-DB85-8040-9574-B7C79FC3CD21,Male,1958-06-15,None,Lost,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-09-02,NaT,Clinical only,2003-09-18,10310-0903-72240,10310-0903-72240,115090300076,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
2,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BDB9DFE2-96DE-EC43-8727-591E32CFC64F,Female,1939-07-07,Single,Lost,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-08-28,NaT,CD4 # or %,2003-08-28,10310-0803-72241,10310-0803-72241,115080300051,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
3,Khomas,Windhoek,Katutura Intermediate Hospital,10310,BF615CCF-0F6C-C144-AF1B-B1CE8003DFB5,Male,1975-03-03,None,Deceased,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-08-12,NaT,CD4 # or %,2003-08-12,10310-0803-72242,10310-0803-72242,115080300052,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,None
4,Khomas,Windhoek,Katutura Intermediate Hospital,10310,0FAA88C5-4850-2F45-9FD3-631544F9C7D0,Male,1961-05-07,Married,Active ART,None,...,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,2003-07-08,NaT,Clinical & CD4,2003-08-22,10310-0703-72243,10310-0703-72243,115070300002,D8CEB3C6-D4F4-45A8-B5C8-E44746D3757E,Incomplete


# Get first and last visit ids per patient

In [76]:
fact_visits_confs = {
    "query": "SELECT id, idpatient, visitDate FROM Fup",
    "cols": ['id', 'client_id', 'visit_date' ]
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

fact_visits_data = fetch_file_maker_data(fact_visits_query, fm_connection)
fact_visits_df = pl.DataFrame(fact_visits_data, schema=fact_visits_cols)

max_visit_dates_df = (
    fact_visits_df
    .groupby('client_id')
    .agg(pl.col('visit_date').max().alias('max_visit_date'))
)

min_visit_dates_df = (
    fact_visits_df
    .groupby('client_id')
    .agg(pl.col('visit_date').min().alias('min_visit_date'))
)

latest_visits_df = fact_visits_df.join(
    max_visit_dates_df,
    on=['client_id', 'visit_date'],
    how='inner'
)

first_visits_df = fact_visits_df.join(
    min_visit_dates_df,
    on=['client_id', 'visit_date'],
    how='inner'
)


# Create first_fact_visits dataframe 

In [77]:
# fact_visits_confs = {
#     "query": "SELECT f.idpatient, f.idfacilitycreate, f.visitDate, f.followupDate, f.scheduledDate, f.careModel, f.pregnantstatus, f.Breastfeeding, f.pregnantlmp, f.pregnantedd, f.ctxadherence, f.arvadherence, f.caremodel, r.code, r.line, cc.treatment, cc.dateTreatment, labResult, pf.stiGenitalUlcers, pf.stiVaginalUrethralDischarge, pf.stiScreenResult, f.oiDetail, f.oiOther FROM Fup f LEFT JOIN patFup pf on f.idPatient=pf.idPatient LEFT JOIN cc ON f.id=cc.idFollowUp LEFT JOIN rgm r ON f.idpatient=r.idpatient",
#     "cols": ['client_id', 'visit_facility_id', 'visit_date', 'next_visit_date', 'scheduled_visit_date', 'care_model', 'pregnancy_status', 'breast_feeding', 'lmp', 'edd', 'ctx_adherence', 'arv_adherence','care_model','current_regimen_code','current_regimen_line', 'cc_treatment_type','cc_treatment_date','cc_results', 'genital_ulcers','vaginal_urethral_discharge','sti_screening_result', 'oi', 'oi_other' ]
# }

# fact_visits_query = fact_visits_confs.get("query")
# fact_visits_cols = fact_visits_confs.get("cols")

# ids = first_visits_df['id'].tolist()

# ids_str = ','.join(f"'{id_}'" for id_ in ids)

# fact_visits_query_updated = f"{fact_visits_query} WHERE f.id IN ({ids_str})"

# fact_visits_data = fetch_file_maker_data(fact_visits_query_updated, fm_connection)
# fact_first_visit_df = pd.DataFrame(fact_visits_data, schema=fact_visits_cols)

# fact_first_visit_df['visit_date'] = pd.to_datetime(fact_first_visit_df['visit_date'], errors='coerce')
# fact_first_visit_df['next_visit_date'] = pd.to_datetime(fact_first_visit_df['next_visit_date'], errors='coerce')

# fact_first_visit_df.head() 

# Add first visit info to fact_sentinel_event

In [78]:
# columns_to_add = {
#     'visit_date': 'first_visit_date', 
#     'visit_facility_id': 'first_visit_facility_id'
# }

# fact_first_visit_df = fact_first_visit_df[list(columns_to_add.keys()) + ['client_id']].rename(schema=columns_to_add)

# fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_visit_df, on='client_id', how='left')
# fact_sentinel_event_df.head()

# Create fact_last_visit dataframe 

In [79]:
fact_visits_confs = {
    "query": "SELECT f.idpatient, f.idfacilitycreate, f.visitDate, f.followupDate, f.scheduledDate, f.careModel, f.pregnantstatus, f.Breastfeeding, f.pregnantlmp, f.pregnantedd, f.ctxadherence, f.arvadherence, f.caremodel, r.code, r.line, cc.treatment, cc.dateTreatment, labResult, pf.stiGenitalUlcers, pf.stiVaginalUrethralDischarge, pf.stiScreenResult, f.oiDetail, f.oiOther FROM Fup f LEFT JOIN patFup pf on f.idPatient=pf.idPatient LEFT JOIN cc ON f.id=cc.idFollowUp LEFT JOIN rgm r ON f.id=r.idFollowUp",
    "cols": ['client_id', 'visit_facility_id', 'visit_date', 'next_visit_date', 'scheduled_visit_date', 'care_model', 'pregnancy_status', 'breast_feeding', 'lmp', 'edd', 'ctx_adherence', 'arv_adherence','care_model','current_regimen_code','current_regimen_line', 'cc_treatment_type','cc_treatment_date','cc_results', 'genital_ulcers','vaginal_urethral_discharge','sti_screening_result', 'oi', 'oi_other' ]
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

ids = latest_visits_df['id'].tolist()

ids_str = ','.join(f"'{id_}'" for id_ in ids)

fact_visits_query_updated = f"{fact_visits_query} WHERE f.id IN ({ids_str})"

fact_visits_data = fetch_file_maker_data(fact_visits_query_updated, fm_connection)
fact_last_visit_df = pl.DataFrame(fact_visits_data, schema=fact_visits_cols)

fact_last_visit_df = fact_last_visit_df.with_columns([
    pl.col('visit_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('visit_date'),
    pl.col('next_visit_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('next_visit_date')
])

fact_last_visit_df.head() 

# Add last visit info to fact_sentinel_event

In [ ]:
columns_to_add = {
    'visit_date': 'last_visit_date',
    'next_visit_date': 'last_next_visit_date', 
    'scheduled_visit_date': 'last_scheduled_visit_date', 
    'pregnancy_status': 'last_pregnancy_status', 
    'breast_feeding': 'last_breast_feeding', 
    'lmp': 'last_lmp', 
    'edd': 'last_edd',
    'visit_facility_id': 'last_visit_facility_id',
    'cc_treatment_type':'last_cc_treatment_type',
    'cc_treatment_date': 'last_cc_treatment_date',
    'cc_results': 'last_cc_results',
    'oi':'last_oi', 
    'oi_other': 'last_oi_other',
    'current_regimen_line': 'last_regimen_line'
}

fact_last_visit_df = fact_last_visit_df[list(columns_to_add.keys()) + ['client_id']].rename(schema=columns_to_add)

max_last_visit_date = fact_last_visit_df['last_visit_date'].max()
print(f'last_refresh_date: {max_last_visit_date}')

fact_sentinel_event_df = fact_sentinel_event_df.join(fact_last_visit_df, on='client_id', how='left')


fact_sentinel_event_df['last_visit_duration'] = (fact_sentinel_event_df['last_next_visit_date'].dt.year - fact_sentinel_event_df['last_visit_date'].dt.year) * 12 + (fact_sentinel_event_df['last_next_visit_date'].dt.month - fact_sentinel_event_df['last_visit_date'].dt.month)

fact_sentinel_event_df = fact_sentinel_event_df.with_columns(pl.col('iit_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False))

fact_sentinel_event_df = fact_sentinel_event_df.with_columns((pl.lit(datetime.today().date()) - pl.col('iit_date')).dt.days().alias('iit_duration'))

bins = [0, 3, 6, float('inf')]
labels = ['< 3 MMP', '3-5 MMP', '6+ MMP']

fact_sentinel_event_df = fact_sentinel_event_df.with_columns(pl.cut(fact_sentinel_event_df['last_visit_duration'], bins=bins, labels=labels).alias('last_visit_duration_category'))

fact_sentinel_event_df.head()

# Create fact_tbt dataframe 

In [ ]:
fact_tbt_confs = {
    "query": "SELECT idpatient, idfacilitycreate, category, startDate, duration, stopDateExpected, stopDateActual , registrationNumber , site, siteDetail , outcome FROM tbt",
    "cols": ['client_id', 'tbt_facility_id', 'tbt_category', 'tbt_start_date', 'tbt_duration' ,'tbt_expected_stop_date', 'tbt_actual_stop_date', 'tbt_registration_number', 'tbt_site', 'tbt_site_detail', 'tbt_outcome']
}

fact_tbt_query = fact_tbt_confs.get("query")
fact_tbt_cols = fact_tbt_confs.get("cols")

fact_tbt_data = fetch_file_maker_data(fact_tbt_query, fm_connection)
fact_tbt_df = pl.DataFrame(fact_tbt_data, schema=fact_tbt_cols)

fact_tbt_df.head()

# Add tbt info to fact_sentinel_event

In [ ]:
columns_to_add = ['client_id', 'tbt_facility_id', 'tbt_category', 'tbt_start_date', 'tbt_duration' ,'tbt_expected_stop_date', 'tbt_actual_stop_date', 'tbt_registration_number', 'tbt_site', 'tbt_site_detail', 'tbt_outcome']

fact_tbt_df = fact_tbt_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_tbt_df, on='client_id', how='left')
fact_sentinel_event_df.head()

# Create fact_tpt dataframe 

In [ ]:
fact_tpt_confs = {
    "query": "SELECT idpatient, idfacilitycreate, iptRegimen, startDate, duration, stopDateExpected, stopDateActual , CASE  WHEN stopReason = '1' THEN 'Completed TPT' WHEN stopReason = '2' THEN 'Cough' WHEN stopReason = '3' THEN 'Confirmed TB' WHEN stopReason = '4' THEN 'Hepatitis' WHEN stopReason = '5' THEN 'Neuropathy' WHEN stopReason = '6' THEN 'Poor Adherence' WHEN stopReason = '7' THEN 'Medicine Out Of Stock' WHEN stopReason = '8' THEN stopReasonOther ELSE '' END AS stopReason, status, adherenceDetail FROM ipt",
    "cols": ['client_id', 'tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration' ,'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_stop_reason', 'tpt_status', 'tpt_adherance']
}

fact_tpt_query = fact_tpt_confs.get("query")
fact_tpt_cols = fact_tpt_confs.get("cols")

fact_tpt_data = fetch_file_maker_data(fact_tpt_query, fm_connection)
fact_tpt_df = pd.DataFrame(fact_tpt_data, schema=fact_tpt_cols)

fact_tpt_df = fact_tpt_df.with_columns([
    pl.col('tpt_start_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('tpt_start_date'),
    pl.col('tpt_expected_stop_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('tpt_expected_stop_date'),
    pl.col('tpt_actual_stop_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('tpt_actual_stop_date')
])

fact_tpt_df['tpt_duration_two'] = (
    (fact_tpt_df['tpt_actual_stop_date'].dt.year - fact_tpt_df['tpt_start_date'].dt.year) * 12 +
    (fact_tpt_df['tpt_actual_stop_date'].dt.month - fact_tpt_df['tpt_start_date'].dt.month)
)

max_tpt_dates_df = (
    fact_tpt_df
    .groupby('client_id')
    .agg(pl.col('tpt_start_date').max().alias('max_tpt_start_date'))
)

fact_tpt_df = fact_tpt_df.join(
    max_tpt_dates_df,
    on=['client_id', 'tpt_start_date'],
    how='inner'
)

fact_tpt_df.head()

# Compute TPT status

In [ ]:
fact_tpt_df = fact_tpt_df.with_columns(
    pl.when(pl.col('tpt_actual_stop_date').is_not_null())
    .then(
        pl.when((pl.col('tpt_regimen') == '3HP') & (pl.col('tpt_duration') >= 70))
        .then('Up-to-date on TPT')
        .when(
            ((pl.col('tpt_regimen') == '9H') & (pl.col('tpt_duration') >= 140)) |
            ((pl.col('tpt_regimen') == '6H') & (pl.col('tpt_duration') >= 140))
        )
        .then('Up-to-date on TPT')
        .when((pl.col('tpt_regimen').is_in(['3H', '6H'])))
        .then('Up-to-date on TPT')
        .otherwise(''))
    .when((pl.col('tpt_start_date').is_not_null()) & (pl.col('tpt_actual_stop_date').is_null()))
    .then('Have not completed TPT')
    .otherwise('Never initiated TPT')
    .alias('tpt_status_two')
)

fact_tpt_df.head()

# Add tpt info to fact_sentinel_event

In [ ]:
columns_to_add = ['client_id', 'tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration', 'tpt_duration_two' ,'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_stop_reason', 'tpt_status', 'tpt_status_two', 'tpt_adherance']

fact_tpt_df = fact_tpt_df[columns_to_add]

fact_sentinel_event_df = fact_sentinel_event_df.join(fact_tpt_df,on='client_id',how='left')

fact_sentinel_event_df.head()

# Create fact_lab dataframe 

In [ ]:
fact_lab_confs = {
    "query": "SELECT l.idpatient, l.idfacilitycreate, l.orderDate, l.result, l.resultDate, lm.name, l.status FROM lab l INNER JOIN LabMaster lm ON lm.id=l.idtest",
    "cols": ['client_id', 'lab_facility_id', 'order_date', 'lab_result', 'result_date', 'test_name','status' ]
}

fact_lab_query = fact_lab_confs.get("query")
fact_lab_cols = fact_lab_confs.get("cols")

fact_lab_data = fetch_file_maker_data(fact_lab_query, fm_connection)
fact_lab_df = pl.DataFrame(fact_lab_data, schema=fact_lab_cols)

fact_lab_df = fact_hiv_diagnosis_df.with_columns([
    pl.col('order_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('order_date'),
    pl.col('result_date').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('result_date')
])

fact_lab_df.head()

# Create fact_first_viral_load dataframe

In [ ]:
fact_last_viral_load_tests_df = fact_lab_df[fact_lab_df['test_name'] == 'Viral Load']
fact_first_viral_load_tests_df = fact_last_viral_load_tests_df.sort_values('order_date').groupby('client_id').first().reset_index()

fact_first_viral_load_tests_df['lab_result'] = pd.to_numeric(fact_first_viral_load_tests_df['lab_result'], errors='coerce')

def categorize_result(result):
    if result < 40:
        return 'Suppressed'
    elif 40 <= result <= 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

fact_first_viral_load_tests_df['category'] = fact_first_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_first_viral_load_tests_df.head()

# Create fact_last_viral_load dataframe

In [ ]:
fact_last_viral_load_tests_df = fact_last_viral_load_tests_df.sort_values('order_date').groupby('client_id').last().reset_index()
fact_last_viral_load_tests_df['lab_result'] = pd.to_numeric(fact_last_viral_load_tests_df['lab_result'], errors='coerce')

def categorize_result(result):
    if result < 40:
        return 'Suppressed'
    elif 40 <= result <= 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

fact_last_viral_load_tests_df['category'] = fact_last_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_last_viral_load_tests_df.head()

# Add first viral load info to fact_sentinel_event

In [ ]:
columns_to_add = {
    'order_date': 'first_viral_load_order_date',
    'result_date': 'first_viral_load_result_date', 
    'lab_result': 'first_viral_load_result',
    'lab_facility_id': 'first_viral_load_facility_id',
    'category': 'first_viral_load_result_category',
    'status': 'first_viral_load_result_status',
}

fact_first_viral_load_tests_df = fact_first_viral_load_tests_df[list(columns_to_add.keys()) + ['client_id']].rename(schema=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_viral_load_tests_df, on='client_id', how='left')

fact_sentinel_event_df.head()

# Add last viral load info to fact_sentinel_event

In [ ]:
columns_to_add = {
    'order_date': 'last_viral_load_order_date',
    'result_date': 'last_viral_load_result_date', 
    'lab_result': 'last_viral_load_result',
    'lab_facility_id': 'last_viral_load_facility_id',
    'category': 'last_viral_load_result_category',
    'status': 'last_viral_load_result_status',
}

fact_last_visit_df = fact_last_viral_load_tests_df[list(columns_to_add.keys()) + ['client_id']].rename(schema=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_last_visit_df, on='client_id', how='left')

# Add duration of ART in months

In [ ]:
def diff_in_months(date1, date2):
    if pd.isnull(date1) or pd.isnull(date2):
        return None
    return (date1.year - date2.year) * 12 + date1.month - date2.month

fact_sentinel_event_df['months_since_art_start'] = fact_sentinel_event_df.apply(lambda row: diff_in_months(max_last_visit_date, row['art_start_date']), axis=1)

fact_sentinel_event_df.head()

# Add program status

In [ ]:
def classify_status(row):
    if pd.isna(row['death_date']) and row['patient_status'] != 'Transferred Out':
        if row['last_next_visit_date'] > max_last_visit_date:
            return 'Active on ART'
        elif (max_last_visit_date - row['last_next_visit_date']).days <= 28:
            return 'Active on ART: Missed Appointment'
        elif  29 < (max_last_visit_date - row['last_next_visit_date']).days <= 90:
            return 'IIT: Missed Appointment'
        elif pd.isnull(row['art_start_date']) and row['patient_status'] == 'Opt Out':
            return 'Never initiated on ART'
        elif (max_last_visit_date - row['last_next_visit_date']).days > 90:
            return 'Lost'
    elif pd.isna(row['death_date']):
        return 'Deceased'
    elif row['patient_status'] != 'Transferred Out':
        return 'Transferred Out'
    
fact_sentinel_event_df['client_status'] = fact_sentinel_event_df.apply(classify_status, axis=1)

fact_sentinel_event_df['tx_curr'] = fact_sentinel_event_df['client_status'].apply(
    lambda status: 'Yes' if status in ['Active', 'Active on ART: Missed Appointment'] else 'No'
)

fact_sentinel_event_df.head()

# Add patient ages at different stages cascade

In [ ]:
fact_sentinel_event_df['hiv_confirmation_age'] = (fact_sentinel_event_df['hiv_confirmation_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['hiv_enrollment_age'] = (fact_sentinel_event_df['hiv_enrollment_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['first_viral_load_result_age'] = (fact_sentinel_event_df['first_viral_load_result_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['last_viral_load_result_age'] = (fact_sentinel_event_df['last_viral_load_result_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
# fact_sentinel_event_df['first_visit_age'] = (fact_sentinel_event_df['first_visit_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['last_visit_age'] = (fact_sentinel_event_df['last_visit_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['current_age'] = (pd.to_datetime('today') - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25

fact_sentinel_event_df.head()

# Add ART Duration

In [ ]:
fact_sentinel_event_df['art_duration'] = (
    (max_last_visit_date.year - fact_sentinel_event_df['art_start_date'].dt.year) * 12 +
    (max_last_visit_date.month - fact_sentinel_event_df['art_start_date'].dt.month)
)

fact_sentinel_event_df['art_duration'] = (max_last_visit_date - fact_sentinel_event_df['art_start_date']).dt.days // 365.25

fact_sentinel_event_df.head()

# Days since last visit

In [ ]:
fact_sentinel_event_df['days_since_last_visit'] = (max_last_visit_date - fact_sentinel_event_df['last_visit_date']).dt.days
fact_sentinel_event_df['years_since_last_visit'] = (max_last_visit_date - fact_sentinel_event_df['last_visit_date']).dt.days // 365.25

fact_sentinel_event_df.head()

# Create dim age groups

In [ ]:
bins = [0, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, float('inf')]
labels = ['0-4', '5-9', '10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64', '65+']

dim_age_group_df = pd.DataFrame({
    'current_age': range(1, 101)
})

dim_age_group_df['pepfar_age_group'] = pd.cut(dim_age_group_df['current_age'], bins=bins, labels=labels, right=False)

dim_age_group_df['paeds_adult_age_group'] = dim_age_group_df['current_age'].apply(lambda x: '0-18 yr old (Pediatric)' if x <= 18 else '19+ yr old (Adult)')

def tri_pillar_classify(age):
    if 0 <= age <= 18:
        return '0-18'
    elif age == 19:
        return '19'
    elif 20 <= age <= 39:
        return '20-39'
    else:
        return '39+'
    
dim_age_group_df['tri_pillar_age_group'] = dim_age_group_df['current_age'].apply(tri_pillar_classify)

dim_age_group_df.head(100)

# Create a column that contains current age groups

In [ ]:
fact_sentinel_event_df = fact_sentinel_event_df.merge(
    dim_age_group_df[['current_age', 'pepfar_age_group','paeds_adult_age_group','tri_pillar_age_group']], 
    on='current_age',
    how='left'
)

fact_sentinel_event_df.head()

# Calculate the difference in months between max_last_visit_date and last_viral_load_result_date

In [ ]:
def calculate_months_since_last_viral_load(date):
    return (max_last_visit_date.year - date.year) * 12 + max_last_visit_date.month - date.month

fact_sentinel_event_df['months_since_last_viral_load'] = fact_sentinel_event_df['last_viral_load_result_date'].apply(calculate_months_since_last_viral_load)

fact_sentinel_event_df.head()

# Add VL Eligibility

In [ ]:
def classify_vl_eligibility(row):
    if row['last_viral_load_result'] >= 40 and 3 < row['months_since_last_viral_load'] <= 6:
        return 'Recent (Slightly delayed)'
    elif (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and 3 < row['months_since_last_viral_load'] <= 6:
        return 'Recent (Slightly delayed)'
    elif row['current_age'] > 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and 12 < row['months_since_last_viral_load'] <= 18:
        return 'Recent (Slightly delayed)'
    elif row['current_age'] <= 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and 6 < row['months_since_last_viral_load'] <= 12:
        return 'Recent (Slightly delayed)'
    
    elif row['art_start_date'] <= max_last_visit_date - pd.DateOffset(days=180):
        if row['last_viral_load_result'] >= 40 and row['months_since_last_viral_load'] <= 3:
            return 'Monitored per Guidelines'
        elif (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['months_since_last_viral_load'] <= 3:
            return 'Monitored per Guidelines'
        elif row['current_age'] <= 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] <= 6:
            return 'Monitored per Guidelines'
        elif row['current_age'] > 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] <= 12:
            return 'Monitored per Guidelines'
    
    elif row['last_viral_load_result'] >= 40 and row['months_since_last_viral_load'] > 6:
        return 'Delayed'
    elif (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['months_since_last_viral_load'] > 6:
        return 'Delayed'
    elif row['current_age'] > 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] > 18:
        return 'Delayed'
    elif row['current_age'] <= 18 and not (row['last_breast_feeding'] == 'yes' or row['last_pregnancy_status'] == 'yes') and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] > 12:
        return 'Delayed'
    
    return 'Unclassified'  

fact_sentinel_event_df['vl_eligibility'] = fact_sentinel_event_df.apply(classify_vl_eligibility, axis=1)

fact_sentinel_event_df.head()

# Add reason for next VL

In [ ]:
def categorize_vl_reason(row):
    if row['months_since_art_start'] is not None and row['months_since_art_start'] > 6 and pd.notnull(row['art_start_date']) and pd.isnull(row['first_viral_load_result_date']):
        return 'First Test'
    elif row['months_since_last_viral_load'] > 3 and row['last_pregnancy_status'] == 'Yes':
        return 'Pregnant Women'
    elif row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] == 'Yes':
        return 'BF Women'
    elif row['months_since_last_viral_load'] > 3 and row['last_viral_load_result_category'] == 'LLV':
        return 'Low Level Viraemia'
    elif row['months_since_last_viral_load'] > 6 and row['last_visit_age'] <= 19:
        return 'Children And Adolescents Under 19'
    elif row['months_since_last_viral_load'] > 12 and row['last_visit_age'] > 19:
        return 'Treatment Monitoring For Adults'
    else:
        return None

fact_sentinel_event_df['reason_for_next_vl'] = fact_sentinel_event_df.apply(categorize_vl_reason, axis=1)

fact_sentinel_event_df.head()

# Add next VL date

In [ ]:
def calculate_next_vl_date(row):
    if pd.isnull(row['last_viral_load_result_date']):
        return None
    elif row['reason_for_next_vl'] == 'First Test':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Pregnant Women':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=3)
    elif row['reason_for_next_vl'] == 'BF Women':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=3)
    elif row['reason_for_next_vl'] == 'Low Level Viraemia':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Children And Adolescents Under 19':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Treatment Monitoring For Adults':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=12)
    else:
        return None

fact_sentinel_event_df['next_vl_date'] = fact_sentinel_event_df.apply(calculate_next_vl_date, axis=1)

print(fact_sentinel_event_df)

# Compute if patient has ever been initiated on ART  

In [ ]:
def determine_has_been_initiated_on_art(row):
    return 'Yes' if pd.notna(row['art_start_date']) else 'No'
    
fact_sentinel_event_df['has_ever_been_initiated_on_art'] = fact_sentinel_event_df.apply(lambda row: determine_has_been_initiated_on_art(row), axis=1)

fact_sentinel_event_df.head()

# Check if last VL is valid

In [ ]:
def is_last_viral_load_valid(row, df):
    df_row = df.iloc[row.name]
    if df_row['months_since_last_viral_load'] <= 12:
        return 'Yes'
    elif pd.notna(df_row['art_start_date']):
        return 'No'
    else:
        return None
    
fact_sentinel_event_df['is_last_viral_load_valid'] = fact_sentinel_event_df.apply(lambda row: is_last_viral_load_valid(row, fact_sentinel_event_df), axis=1)

fact_sentinel_event_df.head()

# Check if patient is virally suppressed

In [ ]:
def determine_is_virally_suppressed(row, df):
    row = df.iloc[row.name]
    if row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result_category'] == 'Suppressed' and row['last_next_visit_date'] >= max_last_visit_date:
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['is_virally_suppressed'] = fact_sentinel_event_df.apply(determine_is_virally_suppressed, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

# Check if patient is not virally suppressed

In [ ]:
def determine_is_not_virally_suppressed(row, df):
    row = df.iloc[row.name]
    if row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= max_last_visit_date:
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['is_not_virally_suppressed'] = fact_sentinel_event_df.apply(determine_is_not_virally_suppressed, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

# Compute needs VL test

In [ ]:
def determine_needs_VL_test(row, df):
    row = df.iloc[row.name]
    if (row['months_since_art_start'] > 6 and pd.notna(row['first_viral_load_result_date']) and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Suppressed' and row['last_next_visit_date'] >= max_last_visit_date and row['months_since_last_viral_load'] > 12 and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= max_last_visit_date and row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['months_since_art_start'] > 3 and pd.notna(row['first_viral_load_result_date']) and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Suppressed' and row['last_next_visit_date'] >= max_last_visit_date and row['months_since_last_viral_load'] > 6 and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= max_last_visit_date and row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 1 and row['is_virally_suppressed'] != 1):
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['needs_vl_test'] = fact_sentinel_event_df.apply(determine_needs_VL_test, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

# Add Six MMP Eligibility

In [ ]:
def determine_six_mmp_eligibility(row, df):
    row = df.iloc[row.name]
    if (row['current_age'] > 2 and (pd.isnull(row['tbt_start_date']) or row['tbt_duration'] >= 6) and (row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result'] < 40) and row['months_since_art_start'] >= 6  and (row['last_regimen_line'] == '1' or row['last_regimen_line'] == '2') and pd.isnull(row['last_oi_other'])):
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['six_mmp_eligible'] = fact_sentinel_event_df.apply(determine_six_mmp_eligibility, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

# Add Six MMP Eligibility But Not Given

In [ ]:
fact_sentinel_event_df['six_mmp_eligible_but_not_given'] = fact_sentinel_event_df.apply(
    lambda row: 'Yes' if row['six_mmp_eligible'] == 'Yes' and row['last_visit_duration_category'] != '6+ MMP' else 'No', 
    axis=1
)

fact_sentinel_event_df.head()

# Compute currrent cascade status

In [ ]:
def determine_art_outcomes(row):
    if row['is_virally_suppressed'] == 'Yes':
        return 'VL Suppressed'
    elif row['is_not_virally_suppressed'] == 'Yes':
        return 'VL Not Suppressed'
    elif row['needs_vl_test'] == 'Yes':
        return 'Needs VL Test'
    elif row['months_since_art_start'] < 6:
        return 'Recently initiated'
    elif row['has_ever_been_initiated_on_art'] == 'No':
        return 'Not initiated on ART'
    elif row['years_since_last_visit'] > 2:
        return 'Case Closed'
    elif row['patient_status'] == 'Deceased':
        return 'Died'
    else:
        return 'Unknown'

fact_sentinel_event_df['art_outcomes'] = fact_sentinel_event_df.apply(determine_art_outcomes, axis=1)

fact_sentinel_event_df.head()

# Change dates from datetime to date 

In [ ]:
fact_sentinel_event_df = fact_sentinel_event_df.apply(lambda col: col.dt.date if col.dtype == 'datetime64[ns]' else col)

fact_sentinel_event_df.head()

# Clean up column names

In [ ]:
def rename_columns(col):
    if '_id' in col:
        return '__' + col
    else:
        col = col.replace('_', ' ').title()
        col = col.replace('Tpt', 'TPT')
        col = col.replace('Tbt', 'TBT')
        col = col.replace('Hiv', 'HIV')
        col = col.replace('Art', 'ART')
        col = col.replace('Iit', 'IIT')
        col = col.replace('Mmp', 'MMP')
        col = col.replace('Pmtct', 'PMTCT')
        col = col.replace('Vl', 'Viral Load')
        col = col.replace('Tx Curr', 'TX Curr')
        col = col.replace('Oi', 'OI')
        col = col.replace('Lmp', 'LMP')
        col = col.replace('Edd', 'EDD')
        col = col.replace('Who', 'WHO')
        col = col.replace('Tb', 'TB')
        return col

fact_sentinel_event_df.columns = [rename_columns(col) for col in fact_sentinel_event_df.columns]

fact_sentinel_event_df.head()

# Define Destination Database Connection

In [ ]:
server = destination_environment.get("server")
database = destination_environment.get("database")
port = destination_environment.get("port")
username = destination_environment.get("username")
password = destination_environment.get("password")

connection_url = ('postgresql://{0}:{1}@{2}:{3}/{4}'
                          .format(username, password, server, port, database))

engine = create_engine(connection_url)

# Write dim age group to table

In [ ]:
table_name = 'dim_age_group'

dim_age_group_df.to_sql(table_name, con=engine, if_exists='replace', index=False, schema='analysis')

dim_age_group_df.head()

# Write sentinel event to table

In [ ]:
table_name = 'fact_sentinel_event'

fact_sentinel_event_df.to_sql(table_name, con=engine, if_exists='replace', index=False, schema='analysis')

fact_sentinel_event_df.head()

# Drop dataframes

In [ ]:
del dim_pat_df
del fact_hiv_diagnosis_df
del fact_hiv_enrolment_df
# del fact_first_visit_df
del fact_last_visit_df
del fact_visits_df
del fact_lab_df
del fact_tpt_df
del fact_meas_df
del fact_first_viral_load_tests_df
del fact_last_viral_load_tests_df
del fact_tbt_df
del fact_tsfr_df

# Log end time

In [ ]:
pd.Timestamp.today()